In [3]:
import pandas as pd
import json
from datetime import datetime
import pyodbc

# Ruta al archivo JSON
json_path = 'C:\\Users\\Fmontenegro\\Documents\\GitHub\\PersonalProjects\\ChatGpt_Powerbi\\conversations.json'  # Cambia esto por la ruta correcta

# Cargar el JSON
with open(json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Verificamos si 'data' es una lista
if not isinstance(data, list):
    raise ValueError("El JSON no tiene la estructura esperada: se esperaba una lista.")

# Función para convertir timestamp
def convert_timestamp(timestamp):
    if timestamp is not None:
        return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    return None

# Inicializamos una lista para almacenar los resultados
all_parts_list = []

# Recorremos cada elemento de la lista 'data'
for item in data:
    if not isinstance(item, dict):
        continue

    title = item.get('title')
    mapping = item.get('mapping', {})
    create_time = item.get('create_time')

    if not isinstance(mapping, dict):
        continue

    for value in mapping.values():
        message = value.get('message')
        if not isinstance(message, dict):
            continue

        content = message.get('content', {})
        parts = content.get('parts', [])

        extracted_parts = []
        for part in parts:
            if isinstance(part, str):
                extracted_parts.append(part)
            elif isinstance(part, dict):
                extracted_parts.append(part.get('text', ''))

        extracted_text = ' '.join(filter(None, extracted_parts))

        if extracted_text:
            all_parts_list.append({
                'title': title,
                'parts': extracted_text,
                'create_time': convert_timestamp(create_time)
            })

# Convertimos la lista a un DataFrame
df = pd.DataFrame(all_parts_list)

# Agrupar por título y reorganizar en pares
result = (
    df.groupby(["title", "create_time"])["parts"]
    .apply(lambda group: pd.DataFrame({
        "pregunta": group.iloc[::2].reset_index(drop=True),
        "respuesta": group.iloc[1::2].reset_index(drop=True)
    }))
    .reset_index(level=[0, 1])
)

# Renombrar las columnas
result.rename(columns={
    'title': 'Encabezado',
    'create_time': 'Fecha',
    'pregunta': 'Pregunta',
    'respuesta': 'Respuesta'
}, inplace=True)

result


,Encabezado,Fecha,Pregunta,Respuesta
0,AI pivotal tech revolution,2024-04-25 11:41:51,"a sentence with ""pivotal"" related with technology",The development of artificial intelligence was...
1,AI pivotal tech revolution,2024-04-25 11:41:51,a sentence with versatile,The new smartphone's versatile features allow ...
2,AI pivotal tech revolution,2024-04-25 11:41:51,another with versatile,Her versatile skills in programming enabled he...
3,AI pivotal tech revolution,2024-04-25 11:41:51,another,The versatile nature of blockchain technology ...
4,AI pivotal tech revolution,2024-04-25 11:41:51,now with to gain footing,The startup leveraged innovative marketing str...
...,...,...,...,...
11,Árboles binarios búsqueda,2024-08-20 15:20:20,### Representación de árboles con vectores\n\n...,NaN
0,Árboles y técnicas de recorrido,2024-10-23 22:56:58,Un árbol es una estructura fundamental en las ...,Aquí tienes las respuestas para las preguntas ...
0,Ética: Carácter y Costumbres,2024-05-31 00:22:33,etica deriva de ethos que tiene dos acepciones,"La palabra ""ética"" proviene del griego ""ethos""..."
1,Ética: Carácter y Costumbres,2024-05-31 00:22:33,que es el termino griego arete,"El término griego ""areté"" (ἀρετή) es uno de lo..."
